In [1]:
%matplotlib inline 
%load_ext autoreload 
%autoreload 2

# A comment

from __future__ import (print_function, division, absolute_import)

import os
import glob
import copy
import sys
import warnings

import numpy as np
np.set_printoptions(threshold='nan')

# Matplotlib related
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cmaps
plt.rc('text', usetex=False)

from astropy.table import \
    Table, \
    Column, \
    vstack, \
    hstack, \
    unique
    
from astropy.utils.metadata import MergeConflictWarning
warnings.filterwarnings('ignore', category=MergeConflictWarning, 
                        append=True)
from astropy import units as u
from astropy.coordinates import SkyCoord as sc
from astropy.utils.console import ProgressBar
from astropy import visualization as vs
from astropy.io import fits

import statistics as stat


plt.rcParams['figure.figsize'] = (12, 7)

In [2]:
table_hsc = Table.read('hsc_s16a_fastlane_g09_short_full.fits', format = 'fits')
table_decals = Table.read ('decals_dr3_g09_short.fits', format = 'fits')

table_hsc.sort('rcmodel_mag')
table_decals.sort('rmag_decal')

ra_hsc = table_hsc['ra']
dec_hsc = table_hsc['dec']
ra_decals = table_decals['RA']
dec_decals = table_decals['DEC']

In [5]:
"""
Absolute magnitude of the Sun in HSC filters
Right now, just use the DES filters
"""
SUN_G = 5.08
SUN_R = 4.62
SUN_I = 4.52
SUN_Z = 4.52
SUN_Y = 4.51

from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import Distance

# Setup the default cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

def luminosity2mag(luminosity, redshift, absmag_sun):
    """
    Convert the luminosity back to absolute magnitude: 
        absmag = absmag_sun - (2.5 * luminosity)
    
    Use redshift to estimate the distance modulus under 
    assumed cosmology.
    More on this topic please read: https://en.wikipedia.org/wiki/Distance_modulus
    
    Here we use the FlatLambdaCDM cosmology, assuming 
        Hubble constant: H0 = 70 km/s/Mpc; 
        Mass density parameter: Omega_m = 0.3
    
    Then convert the absolute magnitude back to apparent
    magnitude:
        mag = absmag + distmod 
    """
    
    # Convert luminosity into absolute magnitude with the 
    # help of the absolute magnitude of Sun in this filter
    absmag = absmag_sun - (2.5 * luminosity) 
    
    # Estimate the distance modulus given redshift 
    # and convert the absolute magnitude back to apparent one
    return (absmag + 
            Distance(z=redshift, cosmology=cosmo).distmod.value)
    
"""
The luminosity2mag() function should work with array
So using the current table, you should be able to convert 
the luminosity within 100 kpc: `lum_100` into magnitude using
"""
mag_i_100kpc = luminosity2mag(table_hsc['lum_100'], 
                              table_hsc['z_best'], 
                              SUN_I)

"""
This magnitude has been corrected for Galactic extinction.
You do not need to do that again. 

We should used mag_r_100, but since we do not have them 
right now, let's using the CModel (r-i) color to convert the 
i-band magnitude into r-band magnitude first. 

Color here describes the magnitude difference between two filters: 
    (g-i)_cmodel = gcmodel_mag - icmodel_mag

For massive galaxies in this sample, their spectral energy
distributions (SED) are quite similar, and using a simple 
color conversion is not a terrible idea.  
""" 

# Get the r-i color using CModel photometry 
ri_cmodel = ((table_hsc['rcmodel_mag'] - table_hsc['a_r']) - 
             (table_hsc['icmodel_mag'] - table_hsc['a_i']))

# Convert the i-band 100 kpc magnitude into r-band assuming
# the same (r-i) colors. 
mag_r_100kpc = mag_i_100kpc + ri_cmodel
    
# Then you can use this magnitude for comparison.